## Real experiments for head movements

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [67]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.dummy import DummyClassifier
from seqlearn.perceptron import StructuredPerceptron
import sklearn_crfsuite
from sklearn_crfsuite import metrics as skcrfmetrics
from scipy.stats import pearsonr,spearmanr

In [68]:
import glob

In [69]:
windowsize="9-11-13"
memory="middle"

PATH = "C:\\Users\\zgk261\\nomco\\"+windowsize+"\\"+memory+"\\"
PATH

'C:\\Users\\zgk261\\nomco\\9-11-13\\middle\\'

In [70]:
!dir /B C:\Users\zgk261\nomco\9-11-13\middle\

F2_M4-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F2_M4-all-final2_primary_M4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F3.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F6.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F1.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F5_F2-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBac

### SKELETON TO READ FILES, SPEAKERS and PRIMARY SPEAKERS


In [71]:
speaker_set = set()

for each_file in glob.glob(PATH+"*.tab"):
    filename=each_file.split("\\")[-1]
    fileinfo=filename.split("-")
    
    primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
    speakers = fileinfo[0]
    l_speaker,r_speaker = speakers.split("_")

    speaker_set.add(primary_speaker)

In [72]:
speaker_set

{'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6'}

### SKELETON TO READ FILES AND COSS_VALIDATE

In the following way: In each iteration, we will leave one speaker out (two files), train a model and test on those two files.

In [73]:

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()


Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Speaker on focus: F3
Training data size: (181651, 28)
Testing data size: (16409, 28)

Speaker on focus: M1
Training data size: (180025, 28)
Testing data size: (18035, 28)

Speaker on focus: M5
Training data size: (180649, 28)
Testing data size: (17411, 28)

Speaker on focus: F4
Training data size: (181137, 28)
Testing data size: (16923, 28)

Speaker on focus: M3
Training data size: (181426, 28)
Testing data size: (16634, 28)

Speaker on focus: F1
Training data size: (181919, 28)
Testing data size: (16141, 28)

Speaker on focus: F6
Training data size: (182822, 28)
Testing data size: (15238, 28)

Speaker on focus: M2
Training data size: (182715, 28)
Testing data size: (15345, 28)

Speaker on focus: F2
Training data size: (181593, 28)


### Experiments:

It seems that in the 3-6-9 experiments considering both the past and future of frames, the MLP was not the best model, but the Logistic Regression. I think, though, that the MLP will show good performance when cross-validating with the whole data.

I will, then, try 3 classifiers:

  * SVM, because it is the default classifier that is commonly used
  * Logistic Regression, because it seems to perform the best in our initial experiments with two dialogue data
  * MLP, because when including more data, I expect to estimate a more general (and thus, better) model

In [74]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
import math

def map_movements(mov_class):
    if mov_class == "Nod" or mov_class == "Jerk":
        return 1
    elif mov_class == "Shake":
        return 2
    elif mov_class!=mov_class: #This returns True if the value is NAN, and it works. It might be source of errors.
        return 0
    else:
        return 3



In [76]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_resultsw = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1score_resultsb = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1_score(y_true, y_pred, average='micro')

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()
    
    
    
    
    
    
    
    #Train data
    features=['PRIMA:velocity-r', 'PRIMA:velocity-clock', 'PRIMA:velocity-x', 'PRIMA:velocity-y'
             ]

    feats_learning = training_data[features]
    X = feats_learning.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    scaler = MinMaxScaler((0,1))
    scaler.fit(X)
    X_norm = scaler.transform(X)
    #X_norm = X
    #We're ignoring anyrow that has a NaN value.
    Y = (training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
    #Y = training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    #Test data
    feats_testing = testing_data[features]

    X_test = feats_testing.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    X_test_norm = scaler.transform(X_test)
    #X_test_norm = X_test
    Y_test = (testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
#    Y_test =  testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    classifiers_to_test = [(DummyClassifier(strategy="most_frequent"),"MF"),
                           (LogisticRegression(solver="liblinear"),"LR"),
                          (LinearSVC(),"LINEARSVC"),
                          (MLPClassifier(hidden_layer_sizes=(30,30,30,30), random_state=442), "MLP")
                          ]
                        #I think that now we have to fine-tune the n_hidden_layers and their sizes (MLP)
    
    
    for clf, clfname in classifiers_to_test:
        print ("Trying classifier ",clfname,speaker_on_focus)
        clf.fit(X_norm, Y)
        y_pred = clf.predict(X_test_norm)
        accuracy_results.loc[clfname,speaker_on_focus] = accuracy_score(Y_test,y_pred)
        #f1score_resultsb.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred)
        f1score_results.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='macro')
        f1score_resultsw.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='weighted')
    
        print (confusion_matrix(Y_test,y_pred))
        print (accuracy_score(Y_test,y_pred)) #same as f1-score (micro-averagaed)
        print (f1_score(Y_test,y_pred, average='macro'))
        print (f1_score(Y_test,y_pred, average='weighted'))
        y_pred=None
        print ()

        
accuracy_results

Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Trying classifier  MF F5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[12154     0]
 [ 4259     0]]
0.7405105708889295
0.42545594567157907
0.6301092504347008

Trying classifier  LR F5
[[11499   655]
 [ 3470   789]]
0.7486748309267044
0.5623057645918491
0.6996898709143219

Trying classifier  LINEARSVC F5
[[11606   548]
 [ 3574   685]]
0.7488576128678487
0.5493262754698414
0.6935712996212257

Trying classifier  MLP F5
[[10773  1381]
 [ 2944  1315]]
0.7364893681837568
0.6054848086206404
0.7148399574629752

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Trying classifier  MF M6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[9316    0]
 [7216    0]]
0.5635131865473022
0.36041473228102755
0.4061969085325493

Trying classifier  LR M6
[[8496  820]
 [5644 1572]]
0.6090007258649891
0.5258237507852143
0.551050736294678

Trying classifier  LINEARSVC M6
[[8548  768]
 [5854 1362]]
0.5994435035083474
0.5061321768384626
0.5334010031683761

Trying classifier  MLP M6
[[7951 1365]
 [4370 2846]]
0.6530970239535446
0.6165316309122559
0.6315732225648919

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Trying classifier  MF M4


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10965     0]
 [ 4380     0]]
0.7145650048875856
0.4167616875712657
0.5956066346326397

Trying classifier  LR M2
[[10667   298]
 [ 3624   756]]
0.7444118605408928
0.5614791178013258
0.6830220610906054

Trying classifier  LINEARSVC M2
[[10756   209]
 [ 3743   637]]
0.7424568263277941
0.5442907985989
0.6732485308446523

Trying classifier  MLP M2
[[9772 1193]
 [2812 1568]]
0.7390029325513197
0.6345416320621807
0.7183882512383118

Speaker on focus: F2
Training data size: (181593, 28)
Testing data size: (16467, 28)

Trying classifier  MF F2


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10315     0]
 [ 6152     0]]
0.6264043237991134
0.38514674034799495
0.48251516690223695

Trying classifier  LR F2
[[9526  789]
 [4057 2095]]
0.7057144592214732
0.630461138679583
0.6726196063711305

Trying classifier  LINEARSVC F2
[[9670  645]
 [4329 1823]]
0.6979413372198944
0.6091981704180819
0.6562783033855784

Trying classifier  MLP F2
[[7936 2379]
 [2455 3697]]
0.7064431894091213
0.685609443940007
0.706069666253742



,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.748675,0.609001,0.676296,0.620391,0.737067,0.609500,0.687821,0.728508,0.737563,0.625935,0.744412,0.705714
LINEARSVC,0.748858,0.599444,0.671209,0.613627,0.732964,0.600540,0.677894,0.728869,0.728394,0.615369,0.742457,0.697941
MLP,0.736489,0.653097,0.698341,0.654275,0.744386,0.664810,0.733381,0.727426,0.761911,0.660585,0.739003,0.706443


In [77]:
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.748675,0.609001,0.676296,0.620391,0.737067,0.609500,0.687821,0.728508,0.737563,0.625935,0.744412,0.705714
LINEARSVC,0.748858,0.599444,0.671209,0.613627,0.732964,0.600540,0.677894,0.728869,0.728394,0.615369,0.742457,0.697941
MLP,0.736489,0.653097,0.698341,0.654275,0.744386,0.664810,0.733381,0.727426,0.761911,0.660585,0.739003,0.706443


In [78]:
f1score_resultsw

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.630109,0.406197,0.509992,0.436293,0.567316,0.391594,0.439536,0.602434,0.539715,0.340577,0.595607,0.482515
LR,0.699690,0.551051,0.617312,0.534256,0.720431,0.526032,0.650049,0.692815,0.695319,0.598804,0.683022,0.672620
LINEARSVC,0.693571,0.533401,0.604066,0.517968,0.712861,0.508117,0.632493,0.684577,0.677889,0.578841,0.673249,0.656278
MLP,0.714840,0.631573,0.682092,0.607623,0.737593,0.628276,0.726457,0.721899,0.747182,0.659429,0.718388,0.706070


In [79]:
f1score_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.425456,0.360415,0.393320,0.370555,0.409328,0.355291,0.371616,0.418514,0.401786,0.336208,0.416762,0.385147
LR,0.562306,0.525824,0.544705,0.488088,0.655953,0.502451,0.620951,0.586144,0.624645,0.597412,0.561479,0.630461
LINEARSVC,0.549326,0.506132,0.526374,0.468752,0.644333,0.482919,0.600308,0.569436,0.600155,0.577190,0.544291,0.609198
MLP,0.605485,0.616532,0.638437,0.575080,0.684583,0.613961,0.712179,0.648967,0.700731,0.659141,0.634542,0.685609


In [80]:
accuracy_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.634612,0.685907,0.679797,0.706679
std,0.075262,0.056079,0.058749,0.039444
min,0.506497,0.609001,0.599444,0.653097
25%,0.582404,0.624549,0.614934,0.663754
50%,0.637360,0.696768,0.687918,0.716934
75%,0.698381,0.737191,0.729892,0.737118
max,0.740511,0.748675,0.748858,0.761911


In [81]:
f1score_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.387033,0.575035,0.556534,0.647937
std,0.028472,0.053367,0.053658,0.041913
min,0.336208,0.488088,0.468752,0.575080
25%,0.368020,0.539985,0.521314,0.615889
50%,0.389234,0.574225,0.559381,0.643702
75%,0.411186,0.621874,0.600193,0.684839
max,0.425456,0.655953,0.644333,0.712179
